In [74]:
import networkx as nx
import pandas as pd
from networkx.drawing.nx_agraph import graphviz_layout
import plotly.plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

#note: for graphviz, need to install it separately, and also install the pygraphviz package.
#also, for plotly need to sign up and set the api key. refer to plotly set up guide on their website

Task 2 Viz 2: Animation of the ranking of the song in different countries, using circles.

Try 1: Using networkx and plotly

In [3]:
rankings_file_path = "../../data/Daily_Ranking.csv"

In [4]:
df = pd.read_csv(rankings_file_path)

In [6]:
#first create dictionary of region code to country name for simplicity later
all_region_codes = ['ec', 'fr', 'ar', 'fi', 'no', 'it', 'lt', 'ph', 'tw', 'nz', 'ee', 'tr', 'us', 'sv', 'cr', 'de', 'cl', 'jp', 'br', 'hn', 'gt', 'ch', 'hu', 'ca', 'pe', 'be', 'my', 'dk', 'bo', 'pl', 'at', 'pt', 'se', 'mx', 'pa', 'uy', 'is', 'es', 'cz', 'ie', 'nl', 'sk', 'co', 'sg', 'id', 'do', 'lu', 'gb', 'py', 'au', 'lv', 'gr', 'hk']
df_countries = pd.read_json("../../data/countries.json")
df_countries = df_countries.transpose()
country_names = {r:df_countries.loc[r.upper()]['name'] for r in all_region_codes}

In [7]:
song = "Shape of You"

In [8]:
df_specific_song = df[df['Track Name'] == song]

In [12]:
print("Select Regions to Show in the Animation")
checkboxes = []
for i in all_region_codes:
    c = widgets.Checkbox(description=country_names[i])
    if i == 'us':
        c.value=True
    checkboxes.append(c)
last = 0
for i in range(1, len(checkboxes)):
    if i%5 == 0 or i == len(checkboxes) - 1:
        display(widgets.HBox(checkboxes[last:i]))
        last = i

Select Regions to Show in the Animation


HBox(children=(Checkbox(value=False, description='Ecuador'), Checkbox(value=False, description='France'), Checkbox(value=False, description='Argentina'), Checkbox(value=False, description='Finland'), Checkbox(value=False, description='Norway')))

HBox(children=(Checkbox(value=False, description='Italy'), Checkbox(value=False, description='Lithuania'), Checkbox(value=False, description='Philippines'), Checkbox(value=False, description='Taiwan'), Checkbox(value=False, description='New Zealand')))

HBox(children=(Checkbox(value=False, description='Estonia'), Checkbox(value=False, description='Turkey'), Checkbox(value=True, description='United States'), Checkbox(value=False, description='El Salvador'), Checkbox(value=False, description='Costa Rica')))

HBox(children=(Checkbox(value=False, description='Germany'), Checkbox(value=False, description='Chile'), Checkbox(value=False, description='Japan'), Checkbox(value=False, description='Brazil'), Checkbox(value=False, description='Honduras')))

...

HBox(children=(Checkbox(value=False, description='Austria'), Checkbox(value=False, description='Portugal'), Checkbox(value=False, description='Sweden'), Checkbox(value=False, description='Mexico'), Checkbox(value=False, description='Panama')))

HBox(children=(Checkbox(value=False, description='Uruguay'), Checkbox(value=False, description='Iceland'), Checkbox(value=False, description='Spain'), Checkbox(value=False, description='Czech Republic'), Checkbox(value=False, description='Ireland')))

HBox(children=(Checkbox(value=False, description='Netherlands'), Checkbox(value=False, description='Slovakia'), Checkbox(value=False, description='Colombia'), Checkbox(value=False, description='Singapore'), Checkbox(value=False, description='Indonesia')))

HBox(children=(Checkbox(value=False, description='Dominican Republic'), Checkbox(value=False, description='Luxembourg'), Checkbox(value=False, description='United Kingdom'), Checkbox(value=False, description='Paraguay'), Checkbox(value=False, description='Australia')))

HBox(children=(Checkbox(value=False, description='Latvia'), Checkbox(value=False, description='Greece')))

In [32]:
# returns the networkx graph for the given set of regions and the date
def make_networkx_graph(regions, date):
    country_grouped = df_specific_song.loc[df_specific_song['Region'].isin(regions)]
    country_grouped = country_grouped.drop(columns=['Track Name', 'Streams', 'URL'])
    country_grouped = country_grouped[country_grouped["Date"]==date]
    
    df_countries = pd.read_json("../../data/countries.json")
    df_countries = df_countries.transpose()
    df_countries = df_countries.drop(columns=['continent', 'capital', 'languages', 'native', 'phone', 'currency'])
    df_countries = df_countries.rename(columns={'name':'Country'})
    df_countries.head()
    country_grouped['Region'] = country_grouped['Region'].str.upper()
    country_grouped = country_grouped.merge(df_countries, how='inner', left_on='Region', right_index=True)
    
    G = nx.Graph()
    unique_countries = country_grouped['Country'].unique().tolist()
    
    for i in unique_countries:
        G.add_node(i, name=i, node_type='Country', position=country_grouped[country_grouped["Country"]==i]["Position"].iloc[0])
        
    return G

In [80]:
#Run this cell to draw the graph/animation!

#extracts the list of days that the chosen song exits on the rankings
song_days = sorted(df_specific_song["Date"].unique().tolist())
if len(song_days) > 100:
    print("truncating to the first hundred days that the song was on the charts")
    song_days = song_days[:100]
# song_days = ['2017-04-08', '2017-04-09', '2017-04-10']
#extracts which regions to use from the checkboxes
regions_to_use = [r for i,r in enumerate(all_region_codes) if checkboxes[i].value]
# regions_to_use = ['us', 'hk', 'ch', 'jp', 'sg']

# make figure
figure = {
    'data': [],
    'layout': {},
    'frames': []
}

# fill in most of layout
figure['layout']['hovermode'] = 'closest'
figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': song_days[0],
    'plotlycommand': 'animate',
    'values': song_days,
    'visible': True
}

figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Day:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

# make initial day data
date = song_days[0]
G = make_networkx_graph(date=date, regions=regions_to_use)
#to calcute the positions for all of the nodes in the graph
# pos = graphviz_layout(G, prog='neato')
pos = nx.drawing.layout.spring_layout(G)

for node in G.nodes(data=True):
    x, y = pos[node[0]]
    data_dict = {
        'x': tuple([x]),
        'y': tuple([y]),
        'mode': 'markers+text',
        'text': tuple(["{}:<br> {}".format(node[1]['name'], node[1]['position'])]),
        'marker': {
            'sizemode': 'area',
            'sizeref': 0.2,
            'size': tuple([(200 - node[1]['position'])])
        },
        'name': node[1]['name']
    }
    figure['data'].append(data_dict)
    
# make frames
for date in song_days:
    frame = {'data': [], 'name': date}
    
    G = make_networkx_graph(date=date, regions=regions_to_use)
    #to calcute the positions for all of the nodes in the graph
#     pos = graphviz_layout(G, prog='neato')
    pos = nx.layout.shell_layout(G)
        
    for node in G.nodes(data=True):
        x, y = pos[node[0]]
        data_dict = {
            'x': tuple([x]),
            'y': tuple([y]),
            'mode': 'markers+text',
            'text': tuple(["{}:<br> {}".format(node[1]['name'], node[1]['position'])]),
            'marker': {
                'sizemode': 'area',
                'sizeref': 0.05,
                'size': tuple([200 - (node[1]['position'])])
            },
            'name': node[1]['name']
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [date],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': date,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

    
figure['layout']['sliders'] = [sliders_dict]

iplot(figure, filename='anim')

truncating to the first hundred days that the song was on the charts
